In [1]:
!git clone https://github.com/tankwin08/Ensemble_model_PySpark.git

fatal: destination path 'Ensemble_model_PySpark' already exists and is not an empty directory.


In [2]:
pip install --upgrade pyspark

/home/naim/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/naim/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Requirement already up-to-date: pyspark in /home/naim/anaconda3/lib/python3.8/site-packages (3.1.2)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark
from pyspark import SparkContext
from pyspark import SQLContext

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
import pyspark.sql.functions as F

In [17]:
from pyspark_iforest.ml.iforest import IForest, IForestModel

In [5]:
sc = SparkContext()
spark = SQLContext(sc)

In [6]:
## data
data_file = '/home/naim/Downloads/spark-iforest/python/Ensemble_model_PySpark/bank-additional/bank-additional-full.csv'

In [7]:
df1 = spark.read.option("delimiter", ";").csv(data_file,header = True, inferSchema = True)

In [8]:
df1

DataFrame[age: int, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, month: string, day_of_week: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, emp.var.rate: double, cons.price.idx: double, cons.conf.idx: double, euribor3m: double, nr.employed: double, y: string]

In [9]:
## rename the column names
df1 = df1.toDF(*(c.replace('.', '_') for c in df1.columns))

In [10]:
## select data type = string
columnList = [item[0] for item in df1.dtypes if item[1].startswith('string')]

In [11]:
df_f = df1.sample(withReplacement=False, fraction=0.6)
df_f2 = df1.subtract(df_f)

In [12]:
fractions = df_f.select("y").distinct().withColumn("fraction", lit(0.5)).rdd.collectAsMap()
df_train = df_f.sampleBy('y',fractions,seed=17)
df_test = df_f.subtract(df_train)

In [13]:
# one hot encoding and assembling
encoding_var = [i[0] for i in df1.dtypes if (i[1]=='string') & (i[0]!='y')]
num_var = [i[0] for i in df1.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

'''from string to interger'''
string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
'''from interger to binary vectors'''
onehot_indexes = [OneHotEncoder(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'y', outputCol = 'label', handleInvalid = 'keep')


## The input for the model should be binary vectors
assembler = VectorAssembler(inputCols = num_var + ['OHE_' + c for c in encoding_var], outputCol = "features")

In [14]:
from pyspark.sql import SparkSession, functions as F, types as T
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from pyspark import SparkConf, SparkContext
np.random.seed(42)
conf = SparkConf()
conf.set('spark.jars', '/home/naim/Downloads/spark-iforest/python/spark-iforest-2.4.0.jar')
spark_session = SparkSession.builder.config(conf=conf).appName('IForest').getOrCreate()

In [15]:
from pyspark_iforest.ml.iforest import *
import sys
from pyspark.sql import SparkSession


In [16]:
iforest = IForest(contamination=0.3, maxDepth=2)

TypeError: 'JavaPackage' object is not callable